
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022
    
Adapted by J. Orjuela (DIADE-IRD) - mai 2023


# <span style="color:#006E7F">__TP3 - CONTIGS QUALITY__ <a class="anchor" id="data"></span>  
    
# <span style="color: #4CACBC;"> 1. Perform a comparison of assemblies using QUAST</span>  


The metrics of assemblies can be evaluated using a quality assessment tool such as [QUAST](http://quast.bioinf.spbau.ru/manual.html).

In part 2, we have assembled raw data using various assemblers and also polished/corrected it. 

Aggregate all assemblies you have produced for your sample in a new folder named "AGGREGATED".

We're going to compare it with QUAST.

### ⚠️ If you had some troubles with precedent assemblies ... 

if you had a problem, you can also download directly AGGREGATED directory from the distant server 

In [4]:
cd ~/work/RESULTS/
wget https://itrop.ird.fr/algae_data/AGGREGATED.tar.gz 
tar -xvf AGGREGATED.tar.gz

AGGREGATED/
AGGREGATED/B8_RAVEN.fasta
AGGREGATED/G11_FLYE.fasta
AGGREGATED/G11_FLYE_MEDAKA.fasta
AGGREGATED/G11_RAVEN.fasta
AGGREGATED/4222_FLYE.fasta
AGGREGATED/4222_RAVEN.fasta
AGGREGATED/B8_FLYE_MEDAKA.fasta
AGGREGATED/4222_FLYE_MEDAKA.fasta
AGGREGATED/4222_RAVEN_MEDAKA.fasta
AGGREGATED/B8_RAVEN_MEDAKA.fasta
AGGREGATED/B8_FLYE.fasta
AGGREGATED/G11_RAVEN_MEDAKA.fasta


### ⚠️ if you want continue with your assemblies please ...

In [ ]:
mkdir -p ~/work/RESULTS/AGGREGATED
cd ~/work/RESULTS/AGGREGATED

### Order assemblies obtained in the samples 

#### sample 4222

In [ ]:
ln -s ~/work/RESULTS/4222_FLYE/assembly.fasta 4222_FLYE.fasta
ln -s ~/work/RESULTS/4222_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/4222_RAVEN/assembly.fasta 4222_RAVEN.fasta
ln -s ~/work/RESULTS/4222_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_RAVEN_MEDAKA.fasta

#### sample B8

In [ ]:
ln -s ~/work/RESULTS/B8_FLYE/assembly.fasta B8_FLYE.fasta
ln -s ~/work/RESULTS/B8_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta B8_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/B8_RAVEN/assembly.fasta B8_RAVEN.fasta
ln -s ~/work/RESULTS/B8_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta B8_RAVEN_MEDAKA.fasta

#### sample G11

In [ ]:
ln -s ~/work/RESULTS/G11_FLYE/assembly.fasta G11_FLYE.fasta
ln -s ~/work/RESULTS/G11_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta G11_FLYE_MEDAKA.fasta
ln -s ~/work/RESULTS/G11_RAVEN/assembly.fasta G11_RAVEN.fasta
ln -s ~/work/RESULTS/G11_RAVEN_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta G11_RAVEN_MEDAKA.fasta

#### check directory with symbolic links

In [5]:
ls -l ~/work/RESULTS/AGGREGATED

total 212496
-rw-r--r-- 1 jovyan users 23001099 May 25 22:41 4222_FLYE.fasta
-rw-r--r-- 1 jovyan users 22633144 May 25 22:42 4222_FLYE_MEDAKA.fasta
-rw-r--r-- 1 jovyan users 22289959 May 25 22:42 4222_RAVEN.fasta
-rw-r--r-- 1 jovyan users 22382505 May 25 22:42 4222_RAVEN_MEDAKA.fasta
-rw-r--r-- 1 jovyan users 15458726 May 25 22:42 B8_FLYE.fasta
-rw-r--r-- 1 jovyan users 15204936 May 25 22:43 B8_FLYE_MEDAKA.fasta
-rw-r--r-- 1 jovyan users 15005713 May 25 22:44 B8_RAVEN.fasta
-rw-r--r-- 1 jovyan users 15095894 May 25 22:44 B8_RAVEN_MEDAKA.fasta
-rw-r--r-- 1 jovyan users 15627449 May 25 22:44 G11_FLYE.fasta
-rw-r--r-- 1 jovyan users 15370900 May 25 22:44 G11_FLYE_MEDAKA.fasta
-rw-r--r-- 1 jovyan users 17694868 May 25 22:45 G11_RAVEN.fasta
-rw-r--r-- 1 jovyan users 17802623 May 25 22:45 G11_RAVEN_MEDAKA.fasta


### <span style="color: #4CACBD;"> 1.1  Run QUAST on assemblies and compare them </span>

In [6]:
cd ~/work/RESULTS/AGGREGATED/
time quast.py *.fasta -o QUAST

/opt/conda/envs/quast/bin/quast.py 4222_FLYE.fasta 4222_FLYE_MEDAKA.fasta 4222_RAVEN.fasta 4222_RAVEN_MEDAKA.fasta B8_FLYE.fasta B8_FLYE_MEDAKA.fasta B8_RAVEN.fasta B8_RAVEN_MEDAKA.fasta G11_FLYE.fasta G11_FLYE_MEDAKA.fasta G11_RAVEN.fasta G11_RAVEN_MEDAKA.fasta -o QUAST

Version: 5.2.0

System information:
  OS: Linux-6.2.0-36-generic-x86_64-with-debian-bookworm-sid (linux_64)
  Python version: 3.7.12
  CPUs number: 20

Started: 2023-11-01 22:28:34

Logging to /home/jovyan/work/RESULTS/AGGREGATED/QUAST/quast.log
NOTICE: Maximum number of threads is set to 5 (use --threads option to set it manually)

CWD: /home/jovyan/work/RESULTS/AGGREGATED
Main parameters: 
  MODE: default, threads: 5, min contig length: 500, min alignment length: 65, min alignment IDY: 95.0, \
  ambiguity: one, min local misassembly length: 200, min extensive misassembly length: 1000

Contigs:
  Pre-processing...
  1  4222_FLYE.fasta ==> 4222_FLYE
  2  4222_FLYE_MEDAKA.fasta ==> 4222_FLYE_MEDAKA
  3  4222_RAVEN.fast

#### Go to the QUAST directory and check the file content.

#### Looking at the output statistics, what are the main differences between the assemblies?

#### Look total sizes, number of contigs, N50 statistics.

#### For each sample what is the best one ?

### <span style="color: #4CACBD;"> 1.2  Comparison of the assemblies against the reference genome of a closely organism </span>

It is possible to compare assemblies against a given reference genome of a closely related organism. 

Use corrected assemblies and compare it with the reference genome

### Compare best algae assemblies against the reference sequence using QUAST

create for that a QUAST_REF directory into the AGGREGATED one

#### Which are the assemblies closest in size relative to the reference ?

#### Does this have the largest N50 and fewest number of contigs?

#### Which assembly does obtain the best contiguity metrics?


## <span style="color: #4CACBD;"> 2. Assessing gene space using BUSCO </span>


Benchmarking Universal Single-Copy Orthologs ([BUSCO](https://busco.ezlab.org/busco_userguide.html)) helps to check if you have a good assembly at the genic level, by searching the expected single-copy lineage-conserved orthologs in any newly-sequenced genome from an appropriate phylogenetic clade.

Calculate the gene space completion for each assembly

In [7]:
mkdir -p ~/work/RESULTS/AGGREGATED/BUSCO
cd ~/work/RESULTS/AGGREGATED/BUSCO

Lineage can be choose in busco database using --list-datasets parameter

In [8]:
# busco env
conda activate busco

In [9]:
busco --list-datasets

2023-11-01 22:29:17 INFO:	Downloading information on latest versions of BUSCO data...
2023-11-01 22:29:19 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/information/lineages_list.2021-12-14.txt.tar.gz'
2023-11-01 22:29:20 INFO:	Decompressing file '/home/jovyan/work/RESULTS/AGGREGATED/BUSCO/busco_downloads/information/lineages_list.2021-12-14.txt.tar.gz'

################################################

Datasets available to be used with BUSCO v4 and v5:

 bacteria_odb10
     - acidobacteria_odb10
     - actinobacteria_phylum_odb10
         - actinobacteria_class_odb10
             - corynebacteriales_odb10
             - micrococcales_odb10
             - propionibacteriales_odb10
             - streptomycetales_odb10
             - streptosporangiales_odb10
         - coriobacteriia_odb10
             - coriobacteriales_odb10
     - aquificae_odb10
     - bacteroidetes-chlorobi_group_odb10
         - bacteroidetes_odb10
             - bacteroidia_odb10
                 

In [10]:
ASSEMBLY="/home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta"
LINEAGE=chlorophyta_odb10
busco -i "$ASSEMBLY" -l "$LINEAGE" -c "${CPUS:-8}" -m genome -o 4222_FLYE_MEDAKA_BUSCO 
# deactivate busco env
conda deactivate

2023-11-01 22:29:20 INFO:	***** Start a BUSCO v5.5.0 analysis, current time: 11/01/2023 22:29:20 *****
2023-11-01 22:29:20 INFO:	Configuring BUSCO with local environment
2023-11-01 22:29:20 INFO:	Mode is genome
2023-11-01 22:29:20 INFO:	Downloading information on latest versions of BUSCO data...
2023-11-01 22:29:23 INFO:	Input file is /home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta
2023-11-01 22:29:23 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/chlorophyta_odb10.2020-08-05.tar.gz'
2023-11-01 22:29:27 INFO:	Decompressing file '/home/jovyan/work/RESULTS/AGGREGATED/BUSCO/busco_downloads/lineages/chlorophyta_odb10.tar.gz'
2023-11-01 22:29:28 INFO:	Running BUSCO using lineage dataset chlorophyta_odb10 (eukaryota, 2020-08-05)
2023-11-01 22:29:28 INFO:	Running 1 job(s) on bbtools, starting at 11/01/2023 22:29:28
2023-11-01 22:29:29 INFO:	[bbtools]	1 of 1 task(s) completed
2023-11-01 22:29:29 INFO:	Running 1 job(s) on metaeuk, starting at 11/01/2023 22:29:

### What does the gene space look like for this assembly?

### Run BUSCO for the best assemblies ...

## <span style="color: #4CACBD;"> 3. Read alignment statistics - remapping</span>

Read congruency is an important measure in determining assembly accuracy. Clusters of read pairs that align incorrectly are strong indicators of mis-assembly.

How well do the reads align back to the draft assemblies? Use minimap2 and samtools to assess the basic alignment statistics.

Make a folder for your results.

We will use 4222_FLYE_MEDAKA.fasta assembly and ONT READS as well ILLUMINA READS.

In [14]:
mkdir -p ~/work/RESULTS/REMAPPING/ONT
cd ~/work/RESULTS/REMAPPING/ONT

In [15]:
ASSEMBLY="/home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta"
ONT="/home/jovyan/work/DATA/ONT/4222_RB2.fastq.gz"

In [16]:
#symbolic link from last assembly in the current repertory REMAPPING
ln -s ${ASSEMBLY} 4222_FLYE_MEDAKA.fasta
# overwrite ASSEMBLY variable
ASSEMBLY=4222_FLYE_MEDAKA.fasta

ln: failed to create symbolic link '4222_FLYE_MEDAKA.fasta': File exists


## I. Mapping on assemblies with ONT reads

In [17]:
minimap2 -ax map-ont -t 4 ${ASSEMBLY} ${ONT} | samtools sort -@ 1 -T "${ASSEMBLY/.fasta/}" -O BAM -o "${ASSEMBLY/.fasta/_ONT_minimap2.bam}" -
samtools index "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"
samtools view -F 0x904 -c "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"

[M::mm_idx_gen::0.349*1.00] collected minimizers
[M::mm_idx_gen::0.424*1.51] sorted minimizers
[M::main::0.424*1.51] loaded/built the index for 31 target sequence(s)
[M::mm_mapopt_update::0.468*1.46] mid_occ = 21
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 31
[M::mm_idx_stat::0.500*1.43] distinct minimizers: 3864622 (94.31% are singletons); average occurrences: 1.099; average spacing: 5.330; total length: 22632780
[M::worker_pipeline::49.538*3.72] mapped 42406 sequences
[M::worker_pipeline::52.963*3.71] mapped 20357 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax map-ont -t 4 4222_FLYE_MEDAKA.fasta /home/jovyan/work/DATA/4222_RB2.fastq.gz
[M::main] Real time: 52.982 sec; CPU: 196.722 sec; Peak RSS: 2.662 GB
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
62394


### What is the percentage of aligned ONT reads in your assembly?

## II. Mapping on assemblies with ILLUMINA reads now !

In [19]:
mkdir -p ~/work/RESULTS/REMAPPING/ILLUMINA
cd ~/work/RESULTS/REMAPPING/ILLUMINA

illumina data is available for only two samples in the distant server : https://itrop.ird.fr/algae_data/ILLUMINA.tar.gz

Don't forget to download it by using `wget` and decompres the directory with `tar` command ! 

You can use minimap2 to align short reads to assemblies!

`minimap2 -ax sr ref.fa read1.fa read2.fa > aln.sam      # short genomic paired-end reads`

In [20]:
cd ~/work/DATA
wget  https://itrop.ird.fr/algae_data/ILLUMINA.tar.gz
tar -xvf ILLUMINA.tar.gz

--2023-11-01 22:41:11--  https://itrop.ird.fr/algae_data/ILLUMINA.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548678690 (523M) [application/x-gzip]
Saving to: ‘ILLUMINA.tar.gz’

ILLUMINA.tar.gz     100%[===================>] 523.26M  34.1MB/s    in 16s     

2023-11-01 22:41:27 (33.2 MB/s) - ‘ILLUMINA.tar.gz’ saved [548678690/548678690]

ILLUMINA/
ILLUMINA/A8_R1.fastq.gz
ILLUMINA/A8_R2.fastq.gz
ILLUMINA/G11_R1.fastq.gz
ILLUMINA/G11_R2.fastq.gz


In [23]:
ASSEMBLY="/home/jovyan/work/RESULTS/AGGREGATED/G11_FLYE_MEDAKA.fasta"
ILLUMINA_R1="/home/jovyan/work/DATA/ILLUMINA/G11_R1.fastq.gz"
ILLUMINA_R2="/home/jovyan/work/DATA/ILLUMINA/G11_R2.fastq.gz"

In [ ]:
cd ~/work/RESULTS/REMAPPING/ILLUMINA

In [24]:
minimap2 -ax sr -t 4 ${ASSEMBLY} ${ILLUMINA_R1} ${ILLUMINA_R2} | samtools sort -@ 1 -T "${ASSEMBLY/.fasta/}" -O BAM -o "${ASSEMBLY/.fasta/_ILL_minimap2.bam}" -
samtools index "${ASSEMBLY/.fasta/_ILL_minimap2.bam}"
samtools view -F 0x904 -c "${ASSEMBLY/.fasta/_ILL_minimap2.bam}"

[M::mm_idx_gen::0.226*1.01] collected minimizers
[M::mm_idx_gen::0.260*1.38] sorted minimizers
[M::main::0.260*1.38] loaded/built the index for 29 target sequence(s)
[M::mm_mapopt_update::0.260*1.38] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 29
[M::mm_idx_stat::0.276*1.36] distinct minimizers: 2494067 (98.81% are singletons); average occurrences: 1.031; average spacing: 5.976; total length: 15370559
[M::worker_pipeline::3.593*3.73] mapped 200300 sequences
[M::worker_pipeline::6.389*3.95] mapped 200304 sequences
[M::worker_pipeline::9.493*4.03] mapped 200302 sequences
[M::worker_pipeline::12.252*4.08] mapped 200300 sequences
[M::worker_pipeline::14.944*4.11] mapped 200300 sequences
[M::worker_pipeline::17.661*4.12] mapped 200300 sequences
[M::worker_pipeline::20.686*4.11] mapped 200300 sequences
[M::worker_pipeline::29.175*3.22] mapped 155146 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax sr -t 4 /home/jovyan/work/RESULTS/AGGREGATED/G

### What is the percentage of aligned ILLUMINA reads in your assembly?

## <span style="color: #4CACBD;"> [OPTIONAL] Blobtools </span>

During the sequence quality assessment stage we tried to discern whether contamination was present. Sometimes this is not feasible at the read level. By plotting Contig GC content vs Contig Read Coverage we can look for clusters of contigs that share similar coverage. The appearance of multiple clusters can indicate multiple organisms. Occasionally, contigs can also be taxonomically classified, providing further evidence for contaminants.

First we need to download some files

In [18]:
cd ~/work/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/blobtools.tar.gz
tar zxvf blobtools.tar.gz
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/testBacteria.dmnd

--2023-11-01 22:38:24--  https://itrop.ird.fr/ont-training/blobtools.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100114880 (95M) [application/x-gzip]
Saving to: ‘blobtools.tar.gz’

blobtools.tar.gz    100%[===================>]  95.48M  49.4MB/s    in 1.9s    

2023-11-01 22:38:26 (49.4 MB/s) - ‘blobtools.tar.gz’ saved [100114880/100114880]

FINISHED --2023-11-01 22:38:26--
Total wall clock time: 2.1s
Downloaded: 1 files, 95M in 1.9s (49.4 MB/s)
blobtools/
blobtools/taxdump.tar.gz
blobtools/names.dmp
blobtools/nodes.dmp
--2023-11-01 22:38:27--  https://itrop.ird.fr/ont-training/testBacteria.dmnd
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214010272 (204M)
Saving to: ‘testBacteria.dmnd’

testBacteria.dm

Run Blobtools on each assembly. Blobtools requires both a BAM file as input and blast output for the classification step.2.

In [25]:
mkdir -p ~/work/RESULTS/BLOBTOOLS
cd ~/work/RESULTS/BLOBTOOLS

### Blastx using diamond 

Run diamond in blastx mode using assembled contigs vs a pre-formatted diamond bacteria database (protein)

In [26]:
#symbolic link from last assembly in the current repertory BLOBTOOLs
ln -s ~/work/RESULTS/4222_FLYE_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta 4222_FLYE_MEDAKA.fasta

In [27]:
time diamond blastx --query 4222_FLYE_MEDAKA.fasta --db ~/work/DATA/testBacteria.dmnd --outfmt 6 qseqid staxids bitscore qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore --sensitive  --max-target-seqs 1  --evalue 1e-25  --threads 4  --out diamond.csv

diamond v2.1.8.162 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
#Target sequences to report alignments for: 1
Opening the database...  [0.049s]
Database: /home/jovyan/work/DATA/testBacteria.dmnd (type: Diamond database, sequences: 271272, letters: 174523808)
Block size = 2000000000
Opening the input file...  [0.007s]
Opening the output file...  [0s]
Loading query sequences...  [0.126s]
Masking queries...  [0.495s]
Algorithm: Double-indexed
Building query histograms...  [0.659s]
Seeking in database...  [0s]
Loading reference sequences...  [0.225s]
Masking reference...  [1.3s]
Initializing temporary storage...  [0s]
Building reference histograms...  [2.142s]
Allocating b

In [ ]:
head diamond.csv

In [ ]:
pwd

### Run blobtools 

Specifically to Blobtools: * nodes and names from ncbi taxdump database can be download from [here](https://github.com/DRL/blobtools#download-ncbi-taxdump-and-create-nodesdb).

In this training nodes and names are available on the DATA directory

In [30]:
ASSEMBLY="4222_FLYE_MEDAKA.fasta"
BAM=~/work/RESULTS/REMAPPING/ONT/${ASSEMBLY/.fasta/_ONT_minimap2.bam}
DIAMONDX=~/work/RESULTS/BLOBTOOLS/diamond.csv
BLOB_NODES=~/work/DATA/blobtools/nodes.dmp
BLOB_NAMES=~/work/DATA/blobtools/names.dmp

In [33]:
rm nodesDB.txt
blobtools create -i ${ASSEMBLY} -b ${BAM} -t ${DIAMONDX} -o quality --names ${BLOB_NAMES} --nodes ${BLOB_NODES} --db nodesDB.txt

[+] Parsing FASTA - 4222_FLYE_MEDAKA.fasta
[+] Store nodesDB in nodesDB.txt
[%] : 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.33M/2.33M [00:01<00:00, 1.72Mit/s]
[+] Parsing tax0 - /home/jovyan/work/RESULTS/BLOBTOOLS/diamond.csv
[+] Computing taxonomy using taxrule(s) bestsum
[%] : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31.0/31.0 [00:00<00:00, 134kit/s]
[+] Parsing bam0 - /home/jovyan/work/RESULTS/REMAPPING/ONT/4222_FLYE_MEDAKA_ONT_minimap2.bam
[+] -> 100.00 (31/31) of sequences have reads aligned to them.
[+] -> 99.77 (163193/163562) of reads are mapped.
[%] : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31.0/31.0 [00:02<00:00, 10.9it/s]


In [34]:
blobtools view -i quality.blobDB.json --cov -o output;
blobtools plot -i quality.blobDB.json;

[+] Reading BlobDB quality.blobDB.json
[+] 	Loading BlobDB into memory ...
[+] 	Deserialising BlobDB (using 'json' module) (this may take a while, consider installing the 'ujson' module) ...
[+] 	Finished in 0.0004334449768066406s
[+] Preparing view(s) ...
[+] Generating data for view
[%] : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31.0/31.0 [00:00<00:00, 364kit/s]
[+] 	Writing output.4222_FLYE_MEDAKA_ONT_minimap2.bam.cov
[+] Generating data for view
[%] : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31.0/31.0 [00:00<00:00, 288kit/s]
[+] 	Writing output.quality.blobDB.table.txt
[+] Writing output ...
[+] Reading BlobDB quality.blobDB.json
[+] 	Loading BlobDB into memory ...
[+] 	Deserialising BlobDB (using 'json' module) (this may take a while, consider i

#### Is there contamination in the assembly?

#### Do any assemblies show strange clustering?

#### Why might coverage vary across contigs within an assembly?

The Blobplots all indicate a single cluster. Some contigs show fairly high coverage in comparison to the rest of the genome which could be repetitive elements in the genome.